# Pre-processing

In [1]:
# Import libraries
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

This notebook contains a first parsing of the dataset we downloaded fom is-academia. <br>
Those files, contain information about courses and projects in which students were enrolled from 2012 to 2020. <br>
We decided to consider only courses interesting for Master students because during Master there is, usually, a bigger freedom in which courses to enroll.

## 1. Select master courses relevant files

We filter all the files and keep only those which contains the word 'Master'. This first filter is quite euristic but actually give good performance to filter for courses which were attended by at least one Master student. <br>
We also decided to exclude the language courses.

In [2]:
# Directory with the .xls files taken from is-academia public access
data_dir = '../../data/'
xls_dir = data_dir + 'xls/'


# Iterate over files and keep only the master courses files
master_files = []
for file in os.listdir(xls_dir):
    path = xls_dir + file
    if os.path.isdir(path):
        continue
    f = open(path, encoding='latin-1')      
    content = f.read()
    if 'Master' in content and 'Cdl' not in content:
        master_files += [file]

In [3]:
len(master_files) # 3473

3473

## 2. Parse files

In [4]:
file = master_files[143]

In [5]:
# Utility to parse the information about a specific course given at the start of the file
# Usually this info contains the name of the course as well the name of the professor and eventually the TAs
def parse_class_info(class_info):
    class_name = prof = ''
    if len(class_info) > 0:
        class_name = class_info[0].text
    if len(class_info) > 1:
        prof_info = class_info[1].text
        if ':' in prof_info:
            prof = prof_info.split(':')[1]
        # We decided to ignore the info regarding TAs and focus only on profs
        if prof.find('Assistant-e-(s)') != -1: 
            prof = prof[:prof.find('Assistant-e-(s)')]
        prof = prof.split(',')
        prof_lst = [x.strip() for x in prof]
    return class_name, prof_lst
    
    

Examples of group info:
- UNIL - Sciences forensiques, 2018-2019, Semestre printemps
- Génie électrique (edoc), 2018-2019
- Science et ingénierie computationnelles, 2018-2019, Master semestre 4


In [6]:
# For each student/group of students enrolled in a certain class we have info regarding the section, the semester
# and the academic year, this utility will parse those info which are usually in standard format
def parse_group_info(group_info, def_section, def_year, def_sem):
    def_section = def_year = def_sem = ''
    group_info = group_info.replace('Ecole polytechnique fédérale de Lausanne,', '')
    info = group_info.split(',')
    if 'Robotique, contrôle et systèmes intelligents (edoc)' in group_info:
        return 'Robotique, contrôle et systèmes intelligents', group_info[group_info.find('20'):], 'edoc'
    if len(info) == 2:
        section = info[0]
        if 'edoc' in group_info:
            section = section[:section.find('(edoc)')]
        elif 'EDOC' in group_info :
            section = section[:section.find('EDOC')]
        year = info[1]
        sem = 'edoc'        
    elif len(info) == 3:
        section, year, sem = info
    elif len(info) > 3:
        section = group_info[:group_info.find('20') - 2]
        year = group_info[group_info.find('20'):]
        year = year[:year.find(',')]
        sem = group_info.split(',')[-1]
    return section.strip(), year.strip(), sem.strip()

Here we implement a parses for the files we selected. Furthermore, we decided to ignore some other files which do not contains information about courses(e.g semester project).

In [7]:
%%time

# File parser
course_id = 1
enrollment_lst = []
ignored_files = []
for file in master_files: 
    ignoreFile = False
    f = open(xls_dir + file, encoding='latin-1')      
    page_body = f.read()
    soup = BeautifulSoup(page_body, 'html.parser')
    
    # empty values
    class_name = None
    profs = []
    def_section = None
    def_year = None
    def_semester = None
    
    for tr in soup.body.table.findAll('tr',recursive=False):
        class_info = tr.findAll('th', recursive=False)
        # If it contains info for a new class
        if len(class_info) > 0:
            if len(class_info) != 2:
                print('Error: More than two lines descrbing the course (the lines should be course name and prof)')
            # Parse the course general information:
            # - class_name
            # - professors (as list)
            class_name, profs = parse_class_info(class_info)
        else:
            # Student for each class are divided in sub-tables depending on section/semester
            # We can have two possible values:
            # - info for a new subset of students (section,year,semester)
            # - list of students (as table)
            
            # Info for a new subset of students
            if len(tr.findAll('table')) == 0:
                # Parse common info for this group of students
                def_section, def_year, def_semester = parse_group_info(tr.text, def_section, def_year, def_semester)
                
            # Table containing the students
            else:
                for table in tr.findAll('table', recurive=False):
                    # Some files contains info about semester/master project
                    # We decided to ignore those files for now and focus only on courses
                    
                    # Format for files that contains project info
                    if len(table.findAll('table')) > 0:
                        ignoreFile = True
                        break
                    
                    # Format for list of students in class
                    for student in table.findAll('tr'):
                        tds = student.findAll('td')
                        name, student_info, extra = tds[:3]
                        name = name.text
                        if len(extra) > 0:
                            print("Error: " + extra)
                        if ',' in student_info:
                            section, year, semester = parse_group_info(student_info.text, def_section, def_year, def_semester)
                        else:
                            section, year, semester = def_section, def_year, def_semester
                        
                        # Save informations about an enrollment
                        if not ignoreFile:
                            #print(section, year, semester)
                            enrollment_lst += [(file, name, section, year, semester, class_name, profs)]                
    if ignoreFile:
        ignored_files += [file]
#enrollment_lst

CPU times: user 3min 14s, sys: 1.72 s, total: 3min 15s
Wall time: 3min 19s


Even after this cleaning we detected some strange/missing informations on our database. Manually checking some of them we can see the come from the data in is-academia and it is not actually a parsing problem.
Some example of weird formats:
- <b>1563.xls Cours UNIL - Faculté de biologie et de médecine II (automne)  Profs divers * </b> <br>
  The semester is actually listed inside the name of the course (repeated infromation) and the prof is not specified (Profs divers *)   
- <b> 6508.xls Théorie et critique du projet MA1 (Lamunière) ['Lamunière Inès'] </b> <br>
 The year of the course (MA1) and the professor (Lamunière) are repeated in the name of the course. This is due to the fact that the course 'Théorie et critique du projet' is a project course and student should enroll with a specific professor.



In [8]:
# Example file prettify for the project files (aka 3rd field of the entry for a student in table is not empty)
# E.g. file 5184
f = open(xls_dir + '5184.xls', encoding='latin-1')      
page_body = f.read()
soup = BeautifulSoup(page_body, 'html.parser')
pretty_f = open(data_dir + 'pretty/' + '5184.xls', 'w', encoding='latin-1')
pretty_f.write(soup.prettify())

16751

In [9]:
# Print number of ingored file from the parser
len(ignored_files)

289

## 3. Create tables

In [10]:
len(enrollment_lst)

828498

In [11]:
enrollment_lst[:3]

[('892.xls',
  'Bays Emmanuelle',
  'Bioingénierie',
  '2006-2007',
  'Master semestre 2',
  'Biological and physiological transport',
  ['Swartz Melody']),
 ('892.xls',
  'Berlier Guillaume',
  'Bioingénierie',
  '2006-2007',
  'Master semestre 2',
  'Biological and physiological transport',
  ['Swartz Melody']),
 ('892.xls',
  'Cosson Steffen',
  'Bioingénierie',
  '2006-2007',
  'Master semestre 2',
  'Biological and physiological transport',
  ['Swartz Melody'])]

In [12]:
df_enrollment_large = pd.DataFrame(enrollment_lst, columns=['file', 'student_name', 'section', 'year', 'semester', 'course_name', 'profs'])
df_enrollment_large.head(2)

,file,student_name,section,year,semester,course_name,profs
0,892.xls,Bays Emmanuelle,Bioingénierie,2006-2007,Master semestre 2,Biological and physiological transport,[Swartz Melody]
1,892.xls,Berlier Guillaume,Bioingénierie,2006-2007,Master semestre 2,Biological and physiological transport,[Swartz Melody]


List of academic years with the corresponding files (one file for one course more or less):

In [13]:
df_enrollment_large.groupby('year').agg({'file':lambda x: (len(set(x)), set(x))})
#set(df_enrollment_large['year'])

,file
year,
2004-2005,"(865, {2259.xls, 4150.xls, 3984.xls, 2866.xls,..."
2005-2006,"(891, {6828.xls, 4067.xls, 4150.xls, 3984.xls,..."
2006-2007,"(907, {1620.xls, 6828.xls, 4150.xls, 3984.xls,..."
2007-2008,"(917, {6828.xls, 4150.xls, 3984.xls, 2866.xls,..."
2008-2009,"(929, {6828.xls, 4150.xls, 3984.xls, 2866.xls,..."
2009-2010,"(990, {722.xls, 763.xls, 3984.xls, 774.xls, 28..."
2010-2011,"(993, {4176.xls, 6828.xls, 4527.xls, 3984.xls,..."
2011-2012,"(1015, {4176.xls, 6828.xls, 4527.xls, 3984.xls..."
2012-2013,"(1043, {6828.xls, 4527.xls, 3984.xls, 774.xls,..."


List of all semesters names:

In [14]:
df_enrollment_large.groupby('semester').agg({'file':lambda x: (len(set(x)), set(x))})

,file
semester,
Bachelor semestre 1,"(56, {3872.xls, 1156.xls, 3086.xls, 1131.xls, ..."
Bachelor semestre 2,"(66, {2419.xls, 877.xls, 1156.xls, 2397.xls, 4..."
Bachelor semestre 3,"(126, {877.xls, 1156.xls, 5956.xls, 4936.xls, ..."
Bachelor semestre 4,"(149, {5371.xls, 3984.xls, 4797.xls, 1117.xls,..."
Bachelor semestre 5,"(414, {4086.xls, 2055.xls, 4530.xls, 1107.xls,..."
Bachelor semestre 5b,"(5, {2864.xls, 5973.xls, 4692.xls, 6343.xls, 2..."
Bachelor semestre 6,"(420, {4150.xls, 2866.xls, 6641.xls, 6345.xls,..."
Bachelor semestre 6b,"(5, {5974.xls, 2078.xls, 2866.xls, 4693.xls, 6..."
Master semestre 1,"(1349, {722.xls, 4530.xls, 5655.xls, 5262.xls,..."


Some of this sections does not correspond to student sections.

In [15]:
len(df_enrollment_large[['student_name', 'section']]),\
    len(df_enrollment_large[['student_name', 'section']].drop_duplicates())

(828498, 109302)

We will retrieve the section in an euristic way: we will takethe section in which the student has attended the largest number of courses.

In [17]:
# Taken from: https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/
def most_frequent(lst): 
    return max(set(lst), key = lst.count)

In [18]:
df_students = df_enrollment_large[['student_name', 'section']].groupby('student_name')\
        .agg({'section': lambda x: list(x)}).section.apply(most_frequent).to_frame().reset_index()

We did not consider the case in which different people have the same name, from our dataset it would be difficult to distinguish this case as we have no way to differentiate them. <br>
One possible way would be checking if some student has attended an extremely large number of courses or if he attended courses in really different years (e.g. master in 2006 and bachelor course in 2020). Both this ways are quite euristic and we decided, for now, to leave this 

In [19]:
df_tmp_students = df_students
df_tmp_students['student_id'] = df_students.index
df_enrollment_tmp = df_tmp_students.merge(df_enrollment_large, left_on = ['student_name'], right_on = ['student_name'])
df_enrollment_med = df_enrollment_tmp.drop('section_y', axis=1).rename(columns={'section_x' : 'major'})
df_enrollment_med.head(2)

,student_name,major,student_id,file,year,semester,course_name,profs
0,Aabid Fouad,Génie mécanique,0,1693.xls,2011-2012,Master semestre 1,Culture médiatique I,"[Meyer Michaël, Haver Gianni]"
1,Aabid Fouad,Génie mécanique,0,6329.xls,2010-2011,Bachelor semestre 3,Thermodynamique et énergétique I,[Favrat Daniel]


Create course table

In [20]:
df_course = df_enrollment_large[['course_name', 'year']]
df_course = df_course.drop_duplicates().reset_index().drop('index',axis=1)
df_course[df_course['course_name'] == 'Data visualization']

,course_name,year
12583,Data visualization,2018-2019
12584,Data visualization,2019-2020
12585,Data visualization,2017-2018


In [21]:
df_course = df_course.reset_index().rename(columns={'index': 'course_id'})
df_course.head()

,course_id,course_name,year
0,0,Biological and physiological transport,2006-2007
1,1,Biological and physiological transport,2007-2008
2,2,Special topics in reactor physics,2015-2016
3,3,Special topics in reactor physics,2014-2015
4,4,Special topics in reactor physics,2013-2014


<b>Create Teaching table:</b> course_id, prof <br>
N.B. There will be multiple tuple with the same course_id in df_teaching because one class can be taught by more than one prof

In [22]:
df_tmp = df_course.merge(df_enrollment_med, left_on = ['year', 'course_name'], right_on =  ['year', 'course_name'])
df_tmp.head()

,course_id,course_name,year,student_name,major,student_id,file,semester,profs
0,0,Biological and physiological transport,2006-2007,Bays Emmanuelle,Ingénierie des sciences du vivant,2959,892.xls,Master semestre 2,[Swartz Melody]
1,0,Biological and physiological transport,2006-2007,Berlier Guillaume,Bioingénierie,3692,892.xls,Master semestre 2,[Swartz Melody]
2,0,Biological and physiological transport,2006-2007,Cosson Steffen,Ingénierie des sciences du vivant,9206,892.xls,Master semestre 2,[Swartz Melody]
3,0,Biological and physiological transport,2006-2007,De Pablo Pena Albora,Ingénierie des sciences du vivant,10336,892.xls,Master semestre 2,[Swartz Melody]
4,0,Biological and physiological transport,2006-2007,Keller Marc,Ingénierie des sciences du vivant,21467,892.xls,Master semestre 2,[Swartz Melody]


In [23]:
df_enrollment = df_tmp[['student_id', 'course_id', 'semester']].drop_duplicates()
df_enrollment.head()

,student_id,course_id,semester
0,2959,0,Master semestre 2
1,3692,0,Master semestre 2
2,9206,0,Master semestre 2
3,10336,0,Master semestre 2
4,21467,0,Master semestre 2


In [24]:
df_teaching = df_tmp[['course_id', 'profs']]
df_teaching = df_teaching.explode('profs').drop_duplicates().rename(columns={'profs':'prof'})
df_teaching.head(2)

,course_id,prof
0,0,Swartz Melody
13,1,Swartz Melody


## 4. Save table

In [25]:
# Save student table
csv_dir = 'csv/'
df_students.to_csv(data_dir + csv_dir + 'student.csv')

In [26]:
# Save courses table
df_course.to_csv(data_dir + csv_dir + 'courses.csv')

In [27]:
# Save teaching table
df_teaching.to_csv(data_dir + csv_dir + 'teaching.csv')

In [28]:
# Save enrollment table
df_enrollment.to_csv(data_dir + csv_dir + 'enrollment.csv')

In [ ]:
df_enrollment_large[df_enrollment_large['student_name'] == '']